# Build (GEGD)
Using an Area of Interest (AOI) from the `aoi` SQLite Database, queries the Global Enchanced GEOINT Delivery API for data over a given time period, builds the `gegd` table from these results, and then populates the table with these values. Other notebooks should be used for adding additional records to the table.

### Import libraries

In [1]:
# Basic stack
from datetime import datetime

# Web Stack
import json
import requests
import getpass

# Database stack
import sqlite3

# Data Science stack
import shapely
from shapely.geometry import box, Polygon
import pandas as pd
import geopandas as gpd
import folium

# Custom stack
import sys; sys.path.append("../../")
from MGP import security, search

### User defined variables

In [2]:
aoi_id = '6'
start_date = '2014-08-13'
end_date = '2024-05-31'
db = "C:/gis/gaia/data/databases/gaia.db"
connect_id = 'eb33ba21-1782-4ffc-8a5c-4854e21effb9'

### User defined functions

In [3]:
def update_gedg(c, iid, column_name, column_value):
    """ Updates GEGD table values of a given column with a given value using the 
            ID as the selection criteria for the update.

        C - A cursor
        ENTITY ID - An ID from  Global Enhanced GEOINT Delivery
        COLUMN NAME - A column name within the EE table to be updated
        COLUMN VALUE - A value to update the column
    """
    force_string_list = ['legacy_id', 'factory_order_number', 'source', 'source_unit',
                        'product_type', 'data_layer', 'legacy_description',
                        'color_band_order', 'asset_name', 'crs_from_pixels',
                        'company_name', 'copyright']
    force_date_list = ['acquisition_date', 'ingest_date']
    force_geom_list = ['geometry']
    
    if column_name in force_string_list:
        sql_string = f"UPDATE gegd SET {column_name} = \"{column_value}\" WHERE id = \"{iid}\""
        c.execute(sql_string)

    elif column_name in force_date_list:
        sql_string = f"UPDATE gegd SET {column_name} = "
        sql_string = sql_string + "? WHERE id = ?"
        c.execute(sql_string, (column_value, iid))
    
    elif column_name in force_geom_list:
        sql_string = f"UPDATE gegd SET {column_name} = GeomFromText(\"{column_value}\") WHERE id = \"{iid}\""
        c.execute(sql_string)
    
    else:
        sql_string = f"UPDATE gegd SET {column_name} = {column_value} WHERE id = \"{iid}\""
        c.execute(sql_string)

def database_activity(db, entity_id, column_name, column_value, activity='update'):
    """ Wrapper function for activities related to database management.
            Currently only supports UPDATING based on  Global Enhanced
            GEOINT Delivery ID values.

        Is dependent on the UPDATE GEGD function above.

        DB - A database target
        ENTITY ID - An  Global Enhanced GEOINT Delivery ID uniquely
            identifying the record of interest.
        COLUMN NAME - The column to be updated
        COLUMN VALUE - The value of the column to be updated
        ACTIVITY - The activity to be carried out. Currently only
            supports UPDATE.
    """
    conn = sqlite3.connect(db)
    conn.enable_load_extension(True)
    conn.execute("SELECT load_extension('mod_spatialite')")
    c = conn.cursor()

    if activity == 'update':
        try:
            update_gedg(c, entity_id, column_name, column_value)
        except Exception as e:
            print("Exception: {} was raised for ID {}".format(e, row['id']))
    else:
        print("Your activity method is not currently supported.")
    
    conn.commit()
    conn.close()

def insert_gedg(c, iid):
    """ Inserts Global Enhanced GEOINT Delivery IDs into the GEGD Data Table.
            The IDs are the Primary Key for the datatable since it is
            unique

        C - A cursor
        ID - An ID value from  Global Enhanced GEOINT Delivery
    """
    sql_string = f"INSERT INTO gegd(id) VALUES (\"{iid}\")"
    c.execute(sql_string)

### Get Log-in details from user
Hint: 1572183293; ???

In [4]:
username = input("Enter your username:  ")
password = getpass.getpass("Enter your password:  ")

Enter your username:   1572183293
Enter your password:   ········


### Retreve the Area of Interest record from the `aoi` table corresponding to the supplied DAR number

In [5]:
conn = sqlite3.connect(db)
conn.enable_load_extension(True)
conn.execute("SELECT load_extension('mod_spatialite')")

c = conn.cursor()

sql_string = f"SELECT aid, name, sqkm, AsText(geom) FROM aoi WHERE aid={aoi_id}"
df = pd.read_sql_query(sql_string, conn)
df = df.rename(columns={'AsText(geom)': 'geometry'}, errors='raise')
df['geometry'] = shapely.wkt.loads(df['geometry'])
gdf_aoi = gpd.GeoDataFrame(df, geometry='geometry')

conn.commit()
conn.close()

gdf_aoi

,aid,name,sqkm,geometry
0,6,Cape Cod Bay,1347.04,"POLYGON ((-70.11938 42.10724, -70.52387 42.109..."


### Build Common Query Language (CQL) spatio-temporal filter

In [6]:
spatial_filter = "INTERSECTS(geometry, {})".format(gdf_aoi['geometry'][0].wkt)
temporal_filter = "acquisitionDate > {} AND acquisitionDate < {}".format(start_date, end_date) 
cql_filter = ' AND '.join([spatial_filter, temporal_filter])
cql_filter

'INTERSECTS(geometry, POLYGON ((-70.119385 42.107239, -70.523865 42.109436, -70.526001 41.747925, -70.121399 41.745544, -70.119385 42.107239))) AND acquisitionDate > 2014-08-13 AND acquisitionDate < 2024-05-31'

### Search MGP

In [8]:
url = 'https://evwhs.digitalglobe.com/catalogservice/wfsaccess'

params = {
    'service': 'WFS',
    'version': '2.0.0',
    'request': 'GetFeature',
    'typeName': 'DigitalGlobe:FinishedFeature',
    'outputFormat': 'application/json',
    'connectId': connect_id,
    'cql_filter': cql_filter
}

results = requests.get(url=url, params=params, auth=(username, password))
r = results.json()
r

{'type': 'FeatureCollection',
 'totalFeatures': 3,
 'features': [{'type': 'Feature',
   'id': '574efe4bff88148b10f9043c889ff154',
   'geometry': {'type': 'Polygon',
    'coordinates': [[[41.98936088, -69.989218],
      [42.078879, -69.98882781],
      [42.12444612, -69.98863615],
      [42.1700126, -69.98843282],
      [42.21586296, -69.98823966],
      [42.26171265, -69.98803468],
      [42.35249841, -69.98764233],
      [42.35268538, -70.01865606],
      [42.35287214, -70.0496697],
      [42.35305868, -70.08068323],
      [42.353245, -70.11169667],
      [42.35343111, -70.14271001],
      [42.353617, -70.17372326],
      [42.35380267, -70.20473641],
      [42.35398813, -70.23574947],
      [42.35417337, -70.26676243],
      [42.3543584, -70.29777529],
      [42.35454321, -70.32878806],
      [42.3547278, -70.35980074],
      [42.35491219, -70.39081332],
      [42.35509636, -70.4218258],
      [42.35528031, -70.45283819],
      [42.35546405, -70.48385048],
      [42.35564758, -70.5148

### Create a GeoDataFrame from query results

In [9]:
df = pd.DataFrame()

for i, record in enumerate(r['features']):
    df.loc[i, 'id'] = record['id']
    df.loc[i, 'aoi_id'] = aoi_id
    df.loc[i, 'legacy_id'] = record['properties']['legacyId']
    df.loc[i, 'factory_order_number'] = record['properties']['factoryOrderNumber']
    df.loc[i, 'acquisition_date'] = record['properties']['acquisitionDate']
    df.loc[i, 'source'] = record['properties']['source']
    df.loc[i, 'source_unit'] = record['properties']['sourceUnit']
    df.loc[i, 'product_type'] = record['properties']['productType']
    df.loc[i, 'cloud_cover'] = record['properties']['cloudCover']
    df.loc[i, 'off_nadir_angle'] = record['properties']['offNadirAngle']
    df.loc[i, 'sun_elevation'] = record['properties']['sunElevation']
    df.loc[i, 'sun_azimuth'] = record['properties']['sunAzimuth']
    df.loc[i, 'ground_sample_distance'] = record['properties']['groundSampleDistance']
    df.loc[i, 'data_layer'] = record['properties']['dataLayer']
    df.loc[i, 'legacy_description'] = record['properties']['legacyDescription']
    df.loc[i, 'color_band_order'] = record['properties']['colorBandOrder']
    df.loc[i, 'asset_name'] = record['properties']['assetName']
    df.loc[i, 'per_pixel_x'] = record['properties']['perPixelX']
    df.loc[i, 'per_pixel_y'] = record['properties']['perPixelY']
    df.loc[i, 'crs_from_pixels'] = record['properties']['crsFromPixels']
    df.loc[i, 'age_days'] = record['properties']['ageDays']
    df.loc[i, 'ingest_date'] = record['properties']['ingestDate']
    df.loc[i, 'company_name'] = record['properties']['companyName']
    df.loc[i, 'copyright'] = record['properties']['copyright']
    df.loc[i, 'niirs'] = record['properties']['niirs']
    df.loc[i, 'geometry'] = Polygon([list(reversed(point)) for point in record['geometry']['coordinates'][0]])
    # df.loc[i, 'geometry'] = Polygon(record['geometry']['coordinates'][0])
gdf = gpd.GeoDataFrame(df, geometry='geometry')
# gdf = gdf.set_geometry("geometry").set_crs(df['crs_from_pixels'][0])
# gdf = gdf.to_crs(4326)
gdf = gdf.set_crs(4326)
gdf

,id,aoi_id,legacy_id,factory_order_number,acquisition_date,source,source_unit,product_type,cloud_cover,off_nadir_angle,...,asset_name,per_pixel_x,per_pixel_y,crs_from_pixels,age_days,ingest_date,company_name,copyright,niirs,geometry
0,574efe4bff88148b10f9043c889ff154,6,1040010089677300,016024927-10,2023-10-01 06:02:38,WV03_VNIR,Strip,Pan Sharpened Natural Color,0.0,7.132186,...,FINISHED,0.322669,-0.322669,EPSG:32738,318.0,2023-10-01 12:34:43,DigitalGlobe,Image Copyright 2023 DigitalGlobe Inc,5.6,"POLYGON ((-69.98922 41.98936, -69.98883 42.078..."
1,bb6b92a5d99696e9d66bdab4a59e3d24,6,10400100812DD200,015585364-10,2023-01-28 06:25:12,WV03_VNIR,Strip,Pan Sharpened Natural Color,0.0,29.738827,...,FINISHED,0.412187,-0.412187,EPSG:32738,564.0,2023-01-28 12:37:37,DigitalGlobe,Image Copyright 2023 DigitalGlobe Inc,5.3,"POLYGON ((-69.98398 41.90745, -69.98078 41.962..."
2,730d3f7f63c582057b1c1cace278d79f,6,1040010082CBCF00,015585344-10,2023-01-28 06:23:49,WV03_VNIR,Strip,Pan Sharpened Natural Color,0.0,20.657362,...,FINISHED,0.357087,-0.357087,EPSG:32738,564.0,2023-01-28 11:54:30,DigitalGlobe,Image Copyright 2023 DigitalGlobe Inc,5.5,"POLYGON ((-69.98693 41.93611, -69.98701 41.983..."


In [10]:
gdf.shape

(3, 26)

### Drop table
This is for demonstration purposes

In [11]:
conn = sqlite3.connect(db)
conn.enable_load_extension(True)
conn.execute("SELECT load_extension('mod_spatialite')")

c = conn.cursor()
c.execute('''DROP TABLE IF EXISTS gegd''')
conn.commit()
conn.close()

### Create `gedg` table

In [12]:
conn = sqlite3.connect(db)
conn.enable_load_extension(True)
conn.execute("SELECT load_extension('mod_spatialite')")
# conn.execute("")

c = conn.cursor()

c.execute('''
    CREATE TABLE IF NOT EXISTS gegd(
        id VARCHAR(32) PRIMARY KEY,
        aoi_id VARCHAR(4),
        legacy_id VARCHAR(16),
        factory_order_number VARCHAR(12),
        acquisition_date SMALLDATETIME,
        source VARCHAR(9),
        source_unit VARCHAR(5),
        product_type VARCHAR(36),
        cloud_cover NUMERIC(3,2),
        off_nadir_angle NUMERIC(2, 3),
        sun_elevation NUMERIC(3, 3),
        sun_azimuth NUMERIC(3, 3),
        ground_sample_distance NUMERIC(2, 2),
        data_layer VARCHAR(10),
        legacy_description VARCHAR(12),
        color_band_order VARCHAR(3),
        asset_name VARCHAR(8),
        per_pixel_x NUMERIC(2, 2),
        per_pixel_y NUMERIC(2, 2),
        crs_from_pixels VARCHAR(10),
        age_days NUMERIC(4, 1),
        ingest_date SMALLDATETIME,
        company_name VARCHAR(12),
        copyright VARCHAR(37),
        niirs NUMERIC(1, 1),
        geometry POLYGON,
        FOREIGN KEY (aoi_id)
            REFERENCES aoi(id)
    )
''')

conn.commit()
conn.close()

### Insert Maxar Geospatial Platform IDs into `mgp` table

In [13]:
conn = sqlite3.connect(db)
conn.enable_load_extension(True)
conn.execute("SELECT load_extension('mod_spatialite')")

c = conn.cursor()

for i, row in gdf.iterrows():
    try:
        insert_gedg(c, row['id'])
    except Exception as e:
        print("Exception: {} was raised for ID {}".format(e, row['id']))
print("Done inserting Imagery IDs into table!")
conn.commit()
conn.close()

Done inserting Imagery IDs into table!


### Update `mgp` records from Maxar Geospatial Platform results

In [14]:
columns = gdf.columns[1:]
for i, row in gdf.iterrows():
    iid = row['id']
    print("Updating information for Imagery ID: {}".format(iid))
    for column in columns:
        database_activity(db, iid, column, row[column])

Updating information for Imagery ID: 574efe4bff88148b10f9043c889ff154
Updating information for Imagery ID: bb6b92a5d99696e9d66bdab4a59e3d24
Updating information for Imagery ID: 730d3f7f63c582057b1c1cace278d79f


### Select newly inserted AOIs, make a GeoDataFrame for validation

In [15]:
conn = sqlite3.connect(db)
conn.enable_load_extension(True)
conn.execute("SELECT load_extension('mod_spatialite')")

columns_list = list(gdf.columns[:-1])
columns_str = ', '.join(columns_list)
sql_string = "SELECT {}, AsText(geometry) FROM gegd WHERE id IS NOT NULL".format(columns_str)

df = pd.read_sql_query(sql_string, conn)
df = df.rename(columns={'AsText(geometry)': 'geometry'}, errors='raise')
df['geometry'] = shapely.wkt.loads(df['geometry'])
gdf = gpd.GeoDataFrame(df, geometry='geometry')

conn.commit()
conn.close()

gdf.head()

,id,aoi_id,legacy_id,factory_order_number,acquisition_date,source,source_unit,product_type,cloud_cover,off_nadir_angle,...,asset_name,per_pixel_x,per_pixel_y,crs_from_pixels,age_days,ingest_date,company_name,copyright,niirs,geometry
0,574efe4bff88148b10f9043c889ff154,6,1040010089677300,016024927-10,2023-10-01 06:02:38,WV03_VNIR,Strip,Pan Sharpened Natural Color,0,7.132186,...,FINISHED,0.322669,-0.322669,EPSG:32738,318,2023-10-01 12:34:43,DigitalGlobe,Image Copyright 2023 DigitalGlobe Inc,5.6,"POLYGON ((-69.98922 41.98936, -69.98883 42.078..."
1,bb6b92a5d99696e9d66bdab4a59e3d24,6,10400100812DD200,015585364-10,2023-01-28 06:25:12,WV03_VNIR,Strip,Pan Sharpened Natural Color,0,29.738827,...,FINISHED,0.412187,-0.412187,EPSG:32738,564,2023-01-28 12:37:37,DigitalGlobe,Image Copyright 2023 DigitalGlobe Inc,5.3,"POLYGON ((-69.98398 41.90745, -69.98078 41.962..."
2,730d3f7f63c582057b1c1cace278d79f,6,1040010082CBCF00,015585344-10,2023-01-28 06:23:49,WV03_VNIR,Strip,Pan Sharpened Natural Color,0,20.657362,...,FINISHED,0.357087,-0.357087,EPSG:32738,564,2023-01-28 11:54:30,DigitalGlobe,Image Copyright 2023 DigitalGlobe Inc,5.5,"POLYGON ((-69.98693 41.93611, -69.98701 41.983..."


In [16]:
# Note that the GDF shape matches that from the above
gdf.shape

(3, 26)

### Plot Areas of Interest on an Interactive Map

In [18]:
def style_function(hex_value):
    return {'color': hex_value, 'fillOpacity': 0}

# Add OpenStreetMap as a basemap
map = folium.Map()
folium.TileLayer('openstreetmap').add_to(map)

# Create a GeoJson layer from the response_geojson and add it to the map
folium.GeoJson(
    gdf['geometry'].to_json(),
    style_function = lambda x: style_function('#0000FF')
).add_to(map)

folium.GeoJson(
    gdf_aoi['geometry'].to_json(),
    style_function = lambda x: style_function('#FF0000')
).add_to(map)

# Zoom to collected images
map.fit_bounds(map.get_bounds(), padding=(100, 100))

# Display the map
map

# End